In [2]:
import  tensorflow as tf

In [1]:
url='http://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

In [3]:
pathofzip=tf.keras.utils.get_file('cats_and_dogs.zip',origin=url,extract=True)

68606236/68606236 [==============================] - 0s 0us/step


In [4]:
import os

In [5]:
path=os.path.join(os.path.dirname(pathofzip),'cats_and_dogs_filtered')

In [6]:
train_dir=os.path.join(path,'train')

In [7]:
val_dir=os.path.join(path,'validation')

In [8]:
train_dataset=tf.keras.utils.image_dataset_from_directory(train_dir,shuffle=True,batch_size=32,image_size=(160,160))

Found 2000 files belonging to 2 classes.


In [9]:
val_dataset=tf.keras.utils.image_dataset_from_directory(val_dir,shuffle=True,batch_size=32,image_size=(160,160))

Found 1000 files belonging to 2 classes.


In [10]:
val_batches=tf.data.experimental.cardinality(val_dataset)

In [11]:
test_dataset=val_dataset.take(val_batches//5)

In [12]:
AUTOTUNE=tf.data.AUTOTUNE
train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset=val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset=test_dataset.prefetch(buffer_size=AUTOTUNE)

In [13]:
data_augmentation=tf.keras.Sequential([tf.keras.layers.RandomFlip('horizontal'),
                                       tf.keras.layers.RandomRotation(0.2),])

In [14]:
preprocess_input=tf.keras.applications.mobilenet_v2.preprocess_input

In [15]:
BATCH_SIZE=32
IMG_SIZE=(160,160)
IMG_SHAPE=IMG_SIZE+(3,)

In [16]:
base_model=tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [17]:
base_model.training=False

In [18]:
inputs=tf.keras.Input(shape=(160,160,3))

In [19]:
X=data_augmentation(inputs)

In [20]:
X=preprocess_input(X)

In [21]:
X=base_model(X,training=False)

In [22]:
global_average_layer=tf.keras.layers.GlobalAveragePooling2D()

In [23]:
X=global_average_layer(X)

In [24]:
X=tf.keras.layers.Dropout(0.2)(X)

In [25]:
prediction_layers=tf.keras.layers.Dense(1)

In [26]:
outputs=prediction_layers(X)

In [27]:
model=tf.keras.Model(inputs,outputs)

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
model.fit(train_dataset,epochs=3,validation_data=val_dataset)

Epoch 1/3
63/63 [==============================] - 209s 3s/step - loss: 7.7010 - accuracy: 0.4985 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 2/3
63/63 [==============================] - 186s 3s/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 3/3
63/63 [==============================] - 216s 3s/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
